In [2]:
import tensorflow as tf
tf.__version__

'2.2.0'

In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [3]:
import pandas as pd
import numpy as np

# fix random seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

In [5]:
text_data = pd.read_csv('../speeches_millercenter_preprocessed.csv')

In [6]:
text_data.head(1)

,Transcript,Summary,President,Source,Date,URL,Word Count,Affiliation,Label
0,34th time speak oval office year soon time wan...,"In this broadcast from the Oval Office, Presid...",Ronald Reagan,National Archives,"January 11, 1989",https://millercenter.org/the-presidency/presid...,3289,Republican,0


In [7]:
# train test split
from sklearn.model_selection import train_test_split
train_corpus, test_corpus, train_label_nums, test_label_nums, train_label_names, test_label_names = train_test_split(text_data['Transcript'],
                                         np.array(text_data['Label']),
                                         np.array(text_data['Affiliation']),
                                         test_size=0.2, random_state=42)
train_corpus.shape, test_corpus.shape

((312,), (79,))

In [10]:
# validation and train split
train_corpus, validation_corpus, train_label_nums, validation_label_nums, train_label_names, validation_label_names = train_test_split(train_corpus,
                                         train_label_nums,
                                         train_label_names,
                                         test_size=0.25, random_state=42)
train_corpus.shape, validation_corpus.shape, test_corpus.shape

((130,), (44,), (79,))

In [11]:
# Tokenize training speeches
t = tf.keras.preprocessing.text.Tokenizer(oov_token='<UNK>')
# fit the tokenizer on the documents
t.fit_on_texts(train_corpus)
t.word_index['<PAD>'] = 0

In [12]:
print(max([(k, v) for k, v in t.word_index.items()], key = lambda x:x[1]), 
      min([(k, v) for k, v in t.word_index.items()], key = lambda x:x[1]), 
      t.word_index['<UNK>'])

('esteem', 11550) ('<PAD>', 0) 1


In [15]:
train_sequences = t.texts_to_sequences(train_corpus)
val_sequences = t.texts_to_sequences(validation_corpus)
test_sequences = t.texts_to_sequences(test_corpus)

In [16]:
print("Vocabulary size={}".format(len(t.word_index)))
print("Number of Documents={}".format(t.document_count))

Vocabulary size=11551
Number of Documents=130


In [17]:
# What does the below do? Limit the vector length? And why do we need it?

In [20]:
MAX_SEQUENCE_LENGTH = 500
# pad dataset to a maximum review length in words
X_train = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_val = tf.keras.preprocessing.sequence.pad_sequences(val_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_test = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_train.shape, X_val.shape, X_test.shape

((130, 500), (44, 500), (79, 500))

In [21]:
VOCAB_SIZE = len(t.word_index)
EMBED_SIZE = 300
EPOCHS=100
BATCH_SIZE=128

In [22]:
word2idx = t.word_index
# Word vectors downloaded from https://fasttext.cc/docs/en/english-vectors.html
FASTTEXT_INIT_EMBEDDINGS_FILE = '../wiki-news-300d-1M-subword.vec'


def load_pretrained_embeddings(word_to_index, max_features, embedding_size, embedding_file_path):    
    
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')
    
    embeddings_index = dict(get_coefs(*row.split(" ")) 
                                for row in open(embedding_file_path, encoding="utf8", errors='ignore') 
                                    if len(row)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean, emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    nb_words = min(max_features, len(word_to_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embedding_size))
    
    for word, idx in word_to_index.items():
        if idx >= max_features: 
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[idx] = embedding_vector

    return embedding_matrix

In [23]:
# What does exactly the below? It uses the embeddings to create the corpus vector representation?

In [24]:
ft_embeddings = load_pretrained_embeddings(word_to_index=word2idx, 
                                           max_features=VOCAB_SIZE, 
                                           embedding_size=EMBED_SIZE, 
                                           embedding_file_path=FASTTEXT_INIT_EMBEDDINGS_FILE)
ft_embeddings.shape

/opt/anaconda3/envs/nlp_capstone/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


(11551, 300)

In [25]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Embedding(VOCAB_SIZE, EMBED_SIZE,
                                    weights=[ft_embeddings],
                                    trainable=True,
                                    input_length=MAX_SEQUENCE_LENGTH))

model.add(tf.keras.layers.Conv1D(filters=256, kernel_size=5, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(pool_size=5))

model.add(tf.keras.layers.Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(pool_size=5))

model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=5, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(pool_size=5))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 300)          3465300   
_________________________________________________________________
conv1d (Conv1D)              (None, 500, 256)          384256    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 100, 256)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 100, 128)          163968    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 20, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 20, 64)            41024     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 4, 64)             0

In [ ]:
# What's the validation dataset used for?

In [26]:
# Fit the model
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                      patience=3,
                                      restore_best_weights=True,
                                      verbose=1)

model.fit(X_train, train_label_nums, 
          validation_data=(X_val, validation_label_nums),
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          shuffle=True,
          callbacks=[es],
          verbose=1)

Epoch 1/100
2/2 [==============================] - 0s 149ms/step - loss: 0.6875 - accuracy: 0.6308 - val_loss: 0.6813 - val_accuracy: 0.5682
Epoch 2/100
2/2 [==============================] - 0s 64ms/step - loss: 0.6570 - accuracy: 0.6462 - val_loss: 0.6997 - val_accuracy: 0.5682
Epoch 3/100
2/2 [==============================] - 0s 70ms/step - loss: 0.6540 - accuracy: 0.6462 - val_loss: 0.7038 - val_accuracy: 0.5682
Epoch 4/100
2/2 [==============================] - 0s 63ms/step - loss: 0.6447 - accuracy: 0.6462 - val_loss: 0.6877 - val_accuracy: 0.5682
Epoch 00004: early stopping


In [27]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
predictions = model.predict_classes(X_test, batch_size=2048, verbose=0).ravel()

print("Accuracy: %.2f%%" % (accuracy_score(test_label_nums.astype(int), predictions)*100))
print(classification_report(test_label_nums, predictions))
pd.DataFrame(confusion_matrix(test_label_nums, predictions))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Accuracy: 50.63%
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        39
           1       0.51      1.00      0.67        40

    accuracy                           0.51        79
   macro avg       0.25      0.50      0.34        79
weighted avg       0.26      0.51      0.34        79



/opt/anaconda3/envs/nlp_capstone/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,0,1
0,0,39
1,0,40
